In [2]:
import pandas as pd

/tmp/ipykernel_9224/4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
df = pd.read_csv('/home/cristina/Public/projetos/predict_price/db/teste_indicium_precificacao.csv')

In [4]:
df.head()

,id,nome,host_id,host_name,bairro_group,bairro,latitude,longitude,room_type,price,minimo_noites,numero_de_reviews,ultima_review,reviews_por_mes,calculado_host_listings_count,disponibilidade_365
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
1,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
2,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
3,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0
4,5099,Large Cozy 1 BR Apartment In Midtown East,7322,Chris,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129


In [5]:
q1 = df['price'].quantile(0.25)
q3 = df['price'].quantile(0.75)
IQR = q3 - q1

outlier = df[(df['price'] < q1 + 1.5*IQR)|(df['price'] > q3 + 1.5*IQR)]

df = df.drop(outlier.index)

Como mostrado na análise exploratória dos dados, a coluna ***price*** possuia muitos outliers. Por interferir no treinamento do modelo, eles foram removidos através do uso de variação interquartil(IQR).

In [6]:
df.shape

(4094, 16)

In [7]:
columns = ['id', 'latitude', 'longitude', 'price']

df_sem_outliers = df.loc[:, columns]

In [8]:
df_sem_outliers.shape

(4094, 4)

Da análise exploratória feita anteriormente, conclui-se que as colunas que influenciam no preço são as de localização, ou seja, as colunas 'latitude' e 'longitude'.
Por isso, para facilitar o treinamento do modelo, foi criado um novo dataframe: ***df_sem_outliers***.

In [9]:
df_sem_outliers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4094 entries, 19 to 48871
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4094 non-null   int64  
 1   latitude   4094 non-null   float64
 2   longitude  4094 non-null   float64
 3   price      4094 non-null   int64  
dtypes: float64(2), int64(2)
memory usage: 159.9 KB


Este novo dataframe não há dados faltantes e todos os valores são numéricos, portanto, não será necessário maiores manipulações nos dados.

In [10]:
# transforma df_new em arquivo csv e exporta para a pasta db
path = '../db/'
df_sem_outliers.to_csv(path + "dados_treino.csv", index = False)